In [ ]:
import os
import torch
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix
from utils.graph import draw_confusion_matrix
from utils.configs import config
from utils.data_loader import get_data_loaders, get_test_loader
from utils.model_list import xception
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from openmax.openmax import OpenMax
from openmax.openmax_utils import get_int_labels

In [ ]:
net = xception(config=config)
net.load_state_dict(torch.load(f'model_weights/fold_{config.fold}/model.pth'))

In [ ]:
train_dl, _ = get_data_loaders(config, thresh_mode=True)

In [ ]:
test_dl, _ = get_test_loader(config)

In [ ]:
# Get true labels
y_true = []

for data in test_dl:
    y = data[1].numpy()
    for label in y:
        y_true.append(label)

y_true = np.asarray(y_true)
y_true = get_int_labels(y_true)

In [ ]:
# OpenMax model
openmax = OpenMax(net, config.fold)

In [ ]:
_, thresh = openmax.compute(train_dl)

In [ ]:
y_pred = openmax.predict(test_dl, thresh)

In [ ]:
f1_score(y_true, y_pred, average='macro')

In [ ]:
cm = confusion_matrix(y_true, y_pred, normalize='true')
cm = np.round(cm, 2)

In [ ]:
draw_confusion_matrix(cm, range(17))